<a href="https://colab.research.google.com/github/anis-kaci/deepseek-ft-table-tennis/blob/main/Fine_Tuning_Deepseek_table_tennis_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets sympy wandb
!pip install --no-cache-dir bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 321.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 307.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 294.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 323.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 358.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 303.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 313.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 310.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 344.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 348.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 323.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    U

In [ ]:
!pip install --upgrade transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:
#dataset


In [ ]:
!wandb login

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Currently logged in as: aniskaci10 (aniskaci10-cy-cergy-paris-universit-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb
wandb.init(
    project= "demo-ft-discoach-deepseek",
    config= {
        "learning_rate": 5e-5,
        "architecture": "DeepSeek-R1-Distill-Qwen-1.5B",
        "dataset": "ai_table_tennis_coach_1000_fr_prompt.jsonl",
        "epochs": 2,

    }
)

wandb: Currently logged in as: aniskaci10 (aniskaci10-cy-cergy-paris-universit-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/ai_table_tennis_coach_1000_fr_prompt.jsonl", split="train")

print(len(dataset))

train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

Generating train split: 0 examples [00:00, ? examples/s]

1000


In [ ]:
def tokenize_function(examples):
  combined_texts = [f"{prompt}\n{completion}" for prompt, completion in zip(examples["prompt"], examples["completion"])]
  tokenized = tokenizer(combined_texts, padding=True, truncation=True, max_length=512)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_datasets = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [ ]:
#define a training class
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned_table_tennis",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    fp16=True,
    logging_steps = 10,
    save_steps=100,
    eval_steps=10,
    learning_rate=3e-5,
    logging_dir="./logs",
    report_to="wandb",
    run_name="DeepSeek_FineTuning_Experiment",
)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear8bitLt(in_featur

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_eval_datasets,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
10,8.157200
20,6.888200
30,5.729000
40,4.651500
50,4.167300
60,3.504000
70,3.383300
80,3.273900
90,3.206800
100,2.937300


TrainOutput(global_step=22500, training_loss=0.26541315422058104, metrics={'train_runtime': 6981.4936, 'train_samples_per_second': 6.446, 'train_steps_per_second': 3.223, 'total_flos': 2.75284044288e+16, 'train_loss': 0.26541315422058104, 'epoch': 50.0})

In [ ]:
save_path = "/content/drive/MyDrive/deepseek_finetuned_table_tennis"
model.save_pretrained(save_path)

tokenizer.save_pretrained(save_path)

print({save_path})

{'/content/drive/MyDrive/deepseek_finetuned_table_tennis'}


In [ ]:
from peft import PeftModel


base_model = AutoModelForCausalLM.from_pretrained(model_name)
model=PeftModel.from_pretrained(base_model, save_path)
model= model.merge_and_unload()


final_save_path = "/content/drive/MyDrive/deepseek_finetuned_table_tennis_full"
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)

print({final_save_path})


{'/content/drive/MyDrive/deepseek_finetuned_table_tennis_full'}


In [ ]:
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

In [ ]:
pipe= pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
prompt = "tu es un expert en tennis de table , Comment m'ameliorer?"

In [ ]:
generated_texts = pipe(prompt, max_length=300, num_return_sequences=1)
generated_texts = generated_texts[0]['generated_text']

In [ ]:
print(generated_texts)

tu es un expert en tennis de table , Comment m'ameliorer?,
Etude des effets de modulations du mouvement de ce tige,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacements de mythe,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacement de mythe,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacement de mythe,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacement de mythe,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacement de mythe,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacement de mythe,
Améliorer mon déplacement,
Pouvoir gérer les déplacement,
Construire mon déplacement,
Etudes des déplacement de mythe,
Améliorer mon déplacement,
Pouvoir gérer les dépla

In [ ]:
prompt = "Quel service est efficace contre un joueur agressif ?"

In [ ]:
generated_texts = pipe(prompt, max_length=300, num_return_sequences=1)
generated_texts = generated_texts[0]['generated_text']

In [ ]:
print(generated_texts)

Quel service est efficace contre un joueur agressif ?


Question 34: Quel service est efficace contre un play reactif?
Answer 34: efficace contre un play reactif : un service : la poise : leцour :
 : :
 : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : : :


In [ ]:
prompt = "Pourquoi je rate mon topspin?"
generated_texts = pipe(prompt, max_length=300, num_return_sequences=1)
generated_texts = generated_texts[0]['generated_text']

print(generated_texts)

Pourquoi je rate mon topspin?**

**Answer:** Pour ce que je rate mon topspin, j’soumpie un bilan. Cela te monte en amou(e). La ra(tion(n) + t(e) + p(air) + (1/T )T (e) te( ra(ction(n ) +... ). Cela te(j(t) t(e) j(ie(n ) +... ). La(t) ra(t(n ) +... ). Cela te(r(t(n ) +... ). La(t) ra(t(n ) +... ). Cela te(t( (e( (1/T )T (e) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )
